В любой случайной группе из 23 человек вероятность совпадения дней рождения у двух человек составляет более 50%

In [25]:
import plotly.graph_objs as go
from math import log, exp, lgamma
import numpy as np

def probability(n):
    if n > 365:
        return 1

    log_prob = lgamma(366) - n * log(365) - lgamma(366 - n)

    return 1 - exp(log_prob)

group_sizes = np.arange(1, 101)
probabilities = [probability(n) for n in group_sizes]

fig = go.Figure([go.Scatter(x=group_sizes, y=probabilities)])

fig.update_layout(
    yaxis=dict(tickformat=".0%"),
    xaxis_title="Количество человек в группе",
    yaxis_title="Вероятность совпадения",
)
fig.show()


In [41]:
import pandas as pd
import numpy as np

bd = pd.Series(range(1, 366))

def simulate(n, trials):
    count = 0
    for _ in range(trials):
        group = bd.sample(n, replace=True)
        if np.any(group.duplicated()):
            count += 1
    return count / trials

group_sizes = list(range(10, 101, 10))
results = []

for n in group_sizes:
    sim = simulate(n, 10000)
    results.append((n, sim))

df = pd.DataFrame(results, columns=["Число человек", "Симуляция"])

df


,Число человек,Симуляция
0,10,0.1165
1,20,0.4073
2,30,0.7077
3,40,0.8889
4,50,0.9696
5,60,0.9930
6,70,0.9989
7,80,0.9999
8,90,1.0000
9,100,1.0000


# Аналитическое решение

## Условие 

Монета подбрасывается один раз в воскресенье.

* Если выпадет **орёл (О)**, то Спящую Красавицу разбудят только в **понедельник**, затем дадут ей лекарство, которое стирает воспоминания о пробуждении, и она больше не проснётся.
* Если выпадет **решка (Р)**, то её разбудят **в понедельник** и снова **во вторник**; после каждого пробуждения память о пробуждении стирают, так что при каждом пробуждении она не помнит предыдущих.

Вопрос: при пробуждении (не зная дня недели) какова апостериорная вероятность того, что монета выпала **орлом**?

## Подход «по пробуждениям»

Рассмотрим все «события пробуждения», которые могут случиться:

* О - 1 пробуждение: (О, понедельник)
* Р - 2 пробуждения: (Р, понедельник), (Р, вторник)

Если мы выбираем **случайное пробуждение**, то возможные пробуждения равновесно возникают в повторяющихся экспериментах: на каждое выпадение **орла** приходится 1 пробуждение, на каждое выпадение **решки** - 2 пробуждения. При честной монете относительные частоты пробуждений: 1 для **орла** и 2 для **решки**, итого 3.

То есть:
$P(О | \text{пробуждение}) = \frac{1}{3}$

## Подход «по исходу монеты»

До пробуждения (с учётом априорной информации) вероятность выпадения **орла** равна $\frac{1}{2}$ - подбрасывание монеты честное, а пробуждение само по себе не даёт новой релевантной информации о монете (в том смысле, что априорная вероятность выпадения **орла** остаётся $\frac{1}{2}$). Следовательно, при пробуждении она должна оставаться $\frac{1}{2}$

In [55]:
import numpy as np
import pandas as pd

total_awakenings = 0
heads_awakenings = 0
monday_awakenings = 0
monday_heads = 0

for _ in range(200000):
    coin_heads = np.random.rand() < 0.5
    if coin_heads:
        total_awakenings += 1
        heads_awakenings += 1
        monday_awakenings += 1
        monday_heads += 1
    else:
        total_awakenings += 2
        monday_awakenings += 1

frac_heads_by_awakenings = heads_awakenings / total_awakenings
frac_heads_monday = monday_heads / monday_awakenings

df = pd.DataFrame([{
    "Всего пробуждений": total_awakenings,
    "Пробуждений при орле": heads_awakenings,
    "P(О|случайное пробуждение)": frac_heads_by_awakenings,
    "P(О|понедельник)": frac_heads_monday
}])

df


,Всего пробуждений,Пробуждений при орле,P(О|случайное пробуждение),P(О|понедельник)
0,300079,99921,0.332982,0.499605
